# Performance testing

### Without threading

In [11]:
import cv2
import torch

# testing
from time import time

# setting device on GPU if available, else CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device, '\n')

# Load Model (yolov5s)
model = torch.hub.load('ultralytics/yolov5', 'yolov5s')

# Set confidence treshold
model.conf = 0.1

# Only detect bottles
model.classes = [39]

# test image
frame = cv2.imread("images/bottle.jpg")

# set amount of detections 
DETECTIONS = 1000

# start timer
start = time()

# --------------------- START DETECTING ------------------------ #
for _ in range(DETECTIONS):
	# detect
	results = model(frame)
	if results.pandas().xyxy[0].empty: continue # turn this off to see the true performance (should gain you around a 20% performance improvement)

# stop timer
end = time()
print(f"{DETECTIONS} detections took {end - start} seconds")
print(f"FPS: {DETECTIONS/(end - start)}")

Using cache found in /home/koen/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2022-4-20 torch 1.11.0+cu102 CUDA:0 (NVIDIA GeForce GTX 1070 Ti, 8116MiB)

Fusing layers... 


Using device: cuda 



YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients
Adding AutoShape... 


1000 detections took 9.85591983795166 seconds
FPS: 101.46186418332603


### With threading

In [26]:
import cv2
import torch

# testing
from time import time
from concurrent.futures import ThreadPoolExecutor

# setting device on GPU if available, else CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device, '\n')

# Load Model (yolov5s)
model = torch.hub.load('ultralytics/yolov5', 'yolov5s')

# Set confidence treshold
model.conf = 0.1

# Only detect bottles
model.classes = [39]

# test image
frame = cv2.imread("images/bottle.jpg")

# set amount of detections 
DETECTIONS = 1000

# worker function
def task(detections):
	detections

detections = []

# init thread pool
executor = ThreadPoolExecutor(1)

# start timer
start = time()

# --------------------- START DETECTING ------------------------ #
for _ in range(DETECTIONS):
	# detect
	results = model(frame)
	# after five detections offload to worker threads
	if len(detections) == 100:
		executor.map(task, detections)
		detections.clear()
	# append data to offload
	if not results.pandas().xyxy[0].empty:
		detections.append(results)

# stop timer
end = time()
print(f"{DETECTIONS} detections took {end - start} seconds")
print(f"FPS: {DETECTIONS/(end - start)}")

Using cache found in /home/koen/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2022-4-20 torch 1.11.0+cu102 CUDA:0 (NVIDIA GeForce GTX 1070 Ti, 8116MiB)

Fusing layers... 


Using device: cuda 



YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients
Adding AutoShape... 


1000 detections took 9.919880867004395 seconds
FPS: 100.8076622498774


# Real world performance testing

### Without threading

In [32]:
import cv2
import torch
from copy import copy

# testing
from time import time

# our modules
from detection import Detection

# setting device on GPU if available, else CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device, '\n')

# Load Model (yolov5s)
model = torch.hub.load('ultralytics/yolov5', 'yolov5s')

# Set confidence treshold
model.conf = 0.1

# Only detect bottles
model.classes = [39]

# test image
frame = cv2.imread("images/bottle.jpg")

# set amount of detections 
DETECTIONS = 10000

# start timer
start = time()

# --------------------- START DETECTING ------------------------ #
for i in range(DETECTIONS):
	base_img = copy(frame) # necessary
	# detect
	results = model(frame)
	print(f"FPS: {i/(time() - start)}")

	# Reformat results to pandas dataframe
	df = results.pandas().xyxy[0]

	# If empty continue with next frame
	# else send detection data
	if (df.empty): continue

	lat, lon = 0, 0
	data = Detection(df, lat, lon, base_img)
	res = data.send("http://localhost/api/test", "test")
		
	print(
		f'[ status: {res.status_code} ]' +
		f'[ {data.detectedAt} ]' +
		f'[ object(s): {len(data.objects)} ]' +
		f'[ lat: {data.lat} - lon: {data.lon} ]'
		)

# stop timer
end = time()
print(f"{DETECTIONS} detections took {end - start} seconds")
print(f"FPS: {DETECTIONS/(end - start)}")

Using cache found in /home/koen/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2022-4-20 torch 1.11.0+cu102 CUDA:0 (NVIDIA GeForce GTX 1070 Ti, 8116MiB)

Fusing layers... 


Using device: cuda 



YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients
Adding AutoShape... 


FPS: 0.0
[ status: 404 ][ Mon Jun  6 09:49:03 2022 ][ object(s): 1 ][ lat: 0 - lon: 0 ]
FPS: 36.51316694378912
[ status: 404 ][ Mon Jun  6 09:49:03 2022 ][ object(s): 1 ][ lat: 0 - lon: 0 ]
FPS: 44.76932765485073
[ status: 404 ][ Mon Jun  6 09:49:03 2022 ][ object(s): 1 ][ lat: 0 - lon: 0 ]
FPS: 48.43046179189728
[ status: 404 ][ Mon Jun  6 09:49:03 2022 ][ object(s): 1 ][ lat: 0 - lon: 0 ]
FPS: 49.8698531597408
[ status: 404 ][ Mon Jun  6 09:49:03 2022 ][ object(s): 1 ][ lat: 0 - lon: 0 ]
FPS: 51.27611298020499
[ status: 404 ][ Mon Jun  6 09:49:03 2022 ][ object(s): 1 ][ lat: 0 - lon: 0 ]
FPS: 52.37914893174179
[ status: 404 ][ Mon Jun  6 09:49:03 2022 ][ object(s): 1 ][ lat: 0 - lon: 0 ]
FPS: 53.095624151169964
[ status: 404 ][ Mon Jun  6 09:49:03 2022 ][ object(s): 1 ][ lat: 0 - lon: 0 ]
FPS: 53.802098565894454
[ status: 404 ][ Mon Jun  6 09:49:03 2022 ][ object(s): 1 ][ lat: 0 - lon: 0 ]
FPS: 54.3981646669568
[ status: 404 ][ Mon Jun  6 09:49:03 2022 ][ object(s): 1 ][ lat: 0 - lon

### With threading

In [34]:
import cv2
import torch
from copy import copy

# testing
from time import time
from concurrent.futures import ThreadPoolExecutor

# our modules
from detection import Detection

# setting device on GPU if available, else CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device, '\n')

# Load Model (yolov5s)
model = torch.hub.load('ultralytics/yolov5', 'yolov5s')

# Set confidence treshold
model.conf = 0.1

# Only detect bottles
model.classes = [39]

# test image
frame = cv2.imread("images/bottle.jpg")

# set amount of detections 
DETECTIONS = 10000

# worker function
def task(detection):
	results, lat, lon, base_img = detection
	# if results.pandas().xyxy[0].empty: return
	data = Detection(results.pandas().xyxy[0], lat, lon, base_img)
	res = data.send("http://localhost/api/test", "test")
	print(
		f'[ status: {res.status_code} ]' +
		f'[ {data.detectedAt} ]' +
		f'[ object(s): {len(data.objects)} ]' +
		f'[ lat: {data.lat} - lon: {data.lon} ]'
		)

detections = []

# init thread pool
executor = ThreadPoolExecutor(1)

# start timer
start = time()

# --------------------- START DETECTING ------------------------ #
for i in range(DETECTIONS):
	base_img = copy(frame) # necessary
	# detect
	results = model(frame)
	print(f"FPS: {i/(time() - start)}")
	# after five detections offload to worker threads
	if len(detections) == 10:
		executor.map(task, detections)
		detections.clear()
	# get location
	lat, lon = 0, 0
	# append data to offload
	if not results.pandas().xyxy[0].empty:
		detections.append([results, lat, lon, base_img])

# stop timer
end = time()
print(f"{DETECTIONS} detections took {end - start} seconds")
print(f"FPS: {DETECTIONS/(end - start)}")

Using cache found in /home/koen/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2022-4-20 torch 1.11.0+cu102 CUDA:0 (NVIDIA GeForce GTX 1070 Ti, 8116MiB)

Fusing layers... 


Using device: cuda 



YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients
Adding AutoShape... 


FPS: 0.0
FPS: 45.06612227355754
FPS: 60.557947474047445
FPS: 67.05093199475652
FPS: 72.1461050549356
FPS: 74.91514162115047
FPS: 76.71735465639128
FPS: 79.24760449787577
FPS: 80.85872917951882
FPS: 81.85746998820346
FPS: 83.33490295187629
[ status: 404 ][ Mon Jun  6 09:55:26 2022 ][ object(s): 1 ][ lat: 0 - lon: 0 ]
FPS: 80.71684321941234
[ status: 404 ][ Mon Jun  6 09:55:26 2022 ][ object(s): 1 ][ lat: 0 - lon: 0 ]
[ status: 404 ][ Mon Jun  6 09:55:26 2022 ][ object(s): 1 ][ lat: 0 - lon: 0 ]
[ status: 404 ][ Mon Jun  6 09:55:26 2022 ][ object(s): 1 ][ lat: 0 - lon: 0 ]
FPS: 77.27566226436531
[ status: 404 ][ Mon Jun  6 09:55:26 2022 ][ object(s): 1 ][ lat: 0 - lon: 0 ]
[ status: 404 ][ Mon Jun  6 09:55:26 2022 ][ object(s): 1 ][ lat: 0 - lon: 0 ]
FPS: 76.16430809373948
[ status: 404 ][ Mon Jun  6 09:55:26 2022 ][ object(s): 1 ][ lat: 0 - lon: 0 ]
[ status: 404 ][ Mon Jun  6 09:55:26 2022 ][ object(s): 1 ][ lat: 0 - lon: 0 ]
FPS: 75.05471352282692
[ status: 404 ][ Mon Jun  6 09:55:26 